In [1]:
import AdventOfCode

In [2]:
{-# LANGUAGE OverloadedStrings #-}
import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Attoparsec.Combinator as A

In [3]:
data Damage
    = Cold
    | Fire
    | Slashing
    | Bludgeoning
    | Radiation
    deriving (Eq, Show)

data Units
    = Units
    { unitsNo :: Int
    , unitsHitPoints :: Int
    , unitsWeakTo :: Maybe [Damage]
    , unitsImmuneTo :: Maybe [Damage]
    , unitsAttackType :: Damage
    , unitsAttackPoints :: Int
    , unitsInitiative :: Int
    } deriving (Eq, Show)

t = "1959 units each with 7875 hit points (weak to cold; immune to slashing, bludgeoning) with an attack that does 38 radiation damage at initiative 20"

import Data.Functor (($>))

parseDamage = A.choice
    [ A.string "cold" $> Cold
    , "fire" $> Fire
    , "slashing" $> Slashing
    , "bludgeoning" $> Bludgeoning
    , "radiation" $> Radiation
    ]

parseDamageTypes = parseDamage `A.sepBy1'` (A.string ", ")
    
parseUnits = do
    no <- A.decimal
    A.string " units each with "
    hp <- A.decimal
    A.string " hit points "
    (weakTo, immuneTo) <- A.choice
        [ do
            A.string "("
            wTo <- A.option Nothing $ fmap Just $ "weak to " *> parseDamageTypes
            iTo <- A.option Nothing $ fmap Just $ "; immune to " *> parseDamageTypes
            A.string ") "
            pure (wTo, iTo)
        , do
            A.string "("
            iTo <- A.option Nothing $ fmap Just $ "immune to " *> parseDamageTypes
            wTo <- A.option Nothing $ fmap Just $ "; weak to " *> parseDamageTypes
            A.string ") "
            pure (wTo, iTo)
        , pure (Nothing, Nothing)
        ]
    A.string "with an attack that does "
    ap <- A.decimal
    A.string " "
    at <- parseDamage
    A.string " damage at initiative "
    initiative <- A.decimal
    pure $ Units no hp weakTo immuneTo at ap initiative

In [4]:
import qualified Data.ByteString as BS

input <- dayLines 24

immune = drop 1 $ takeWhile (\c -> BS.length c > 0) input
infection = drop 2 $ dropWhile (\c -> BS.length c > 0) input


immuneUnits = map (parsed parseUnits) immune
infectionUnits = map (parsed parseUnits) infection